# Script to get the location coordinates

## To-Do
- Destination 
1. Get the appropriate location name, port/airport.
2. Get the coordinates
- Source
1. Get the coordinates, if mode is AIR then it should be the airport of that city. If ocean, then the nearest port of that city. If road then that place coordinates

In [1]:
from geopy.geocoders import ArcGIS

import pandas as pd

In [2]:
nom = ArcGIgeocoders

In [11]:
location = nom.geocode('Standard Diagnostics, Korea')

In [12]:
location

Location(Dél-Korea, (36.35627096300004, 127.80639593300009, 0.0))

In [13]:
print(str(location.latitude) + ',' + str(location.longitude))

36.35627096300004,127.80639593300009


In [14]:
str(location.latitude) + ',' + str(location.longitude)

'36.35627096300004,127.80639593300009'

In [15]:
country = nom.reverse(str(location.latitude) + ',' + str(location.longitude))

In [16]:
country

Location(충청북도 옥천군 청산면 교평리, (36.35627096300004, 127.80639593300009, 0.0))

In [62]:
help(country)

Help on Location in module geopy.location object:

class Location(builtins.object)
 |  Location(address, point, raw)
 |  
 |  Contains a parsed geocoder response. Can be iterated over as
 |  ``(location<String>, (latitude<float>, longitude<Float))``.
 |  Or one can access the properties ``address``, ``latitude``,
 |  ``longitude``, or ``raw``. The last
 |  is a dictionary of the geocoder's response for this item.
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, other)
 |      Return self==value.
 |  
 |  __getitem__(self, index)
 |      Backwards compatibility with geopy<0.98 tuples.
 |  
 |  __getstate__(self)
 |  
 |  __init__(self, address, point, raw)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __iter__(self)
 |  
 |  __len__(self)
 |  
 |  __ne__(self, other)
 |      Return self!=value.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  __setstate__(self, state)
 |  
 |  __str__(self)
 |      Return str(self).
 |  
 |  -------------

In [36]:
# Read ports data
import geopandas as gpd
ports = gpd.read_file('Data/attributed_ports.geojson')



,Country,Function,LOCODE,Name,NameWoDiac,Status,outflows,geometry
0,United Arab Emirates,1-345---,AEAUH,Abu Dhabi,Abu Dhabi,AI,4.159714e+04,POINT (54.36667 24.46667)
1,United Arab Emirates,1-------,AERUW,Ar Ruways,Ar Ruways,RL,1.665560e+05,POINT (52.73333 24.11667)
2,United Arab Emirates,1-------,AEKLF,Khor al Fakkan,Khor al Fakkan,RL,7.904065e+05,POINT (56.35000 25.33333)
3,United Arab Emirates,1-3-----,AEMKH,Mina Khalid,Mina Khalid,RL,6.469650e+05,POINT (55.36667 25.35000)
4,United Arab Emirates,1-------,AEKHL,Mina Khalifa/Abu Dhabi,Mina Khalifa/Abu Dhabi,RL,1.834146e+07,POINT (54.66667 24.83333)
...,...,...,...,...,...,...,...,...
851,Yemen,1--45---,YEADE,Aden,Aden,AI,1.260825e+05,POINT (45.03667 12.77944)
852,Yemen,1--4----,YEMKX,Mukalla,Mukalla,AI,3.074500e+04,POINT (49.12424 14.54248)
853,South Africa,1234----,ZAELS,East London,East London,AF,1.560000e+04,POINT (27.91162 -33.01529)
854,South Africa,1--45---,ZAPLZ,Port Elizabeth,Port Elizabeth,AF,2.557154e+06,POINT (25.61494 -33.96109)


In [48]:
ports_df = ports[['Country','Name','geometry']].copy()

In [49]:
ports_df['latitude'] = ports_df.geometry.apply(lambda p: p.x)
ports_df['longitude'] = ports_df.geometry.apply(lambda p: p.y)

In [51]:
ports_df['Country'].unique()

array(['United Arab Emirates', 'Antigua and Barbuda', 'Argentina',
       'American Samoa', 'Australia', 'Aruba', 'Bangladesh', 'Belgium',
       'Bahrain', 'Bonaire, Sint Eustatius and Saba', 'Brazil', 'Bahamas',
       'Belize', 'Canada', 'Congo, The Democratic Republic of the',
       'Congo', "Côte d'Ivoire", 'Chile', 'China', 'Colombia',
       'Costa Rica', 'Cuba', 'Cabo Verde', 'Curaçao', 'Christmas Island',
       'Cyprus', 'Germany', 'Denmark', 'Dominican Republic', 'Algeria',
       'Ecuador', 'Estonia', 'Egypt', 'Eritrea', 'Spain', 'Finland',
       'Fiji', 'Faroe Islands', 'France', 'United Kingdom',
       'French Guiana', 'Greenland', 'Guadeloupe', 'Greece', 'Guatemala',
       'Guam', 'Guinea-Bissau', 'Croatia', 'Haiti', 'Indonesia',
       'Ireland', 'Israel', 'India', 'Iraq', 'Iran, Islamic Republic of',
       'Iceland', 'Italy', 'Jordan', 'Japan', 'Cambodia', 'Comoros',
       'Saint Kitts and Nevis', 'Korea, Republic of', 'Kuwait',
       'Cayman Islands', 'Lebanon'